In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv("/content/train.csv")
test=pd.read_csv("/content/test_actual.csv")
submissions=pd.read_csv("/content/sample_submission.csv")

In [ ]:
x=data["product_name"]+data["review_title"]+data["review_text"]
y=data["review_rating"].values

In [ ]:
df=pd.DataFrame(list(zip(x, y)),columns=['text','labels'])

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.dropna(inplace=True)

In [ ]:
test_data=test["product_name"]+test["review_title"]+test["review_text"]
test_data

0      FRYE Women's Maya Canvas Low Lace Sneaker, Blu...
1      Merrell Men's Jungle Moc Leather Waterproof Ic...
2      adidas Kids&#39; Fortarun Ac Running ShoeVery ...
3      PEARL IZUMI Women's W Elite Road v5 Cycling Sh...
4              CLARKS Men&#39;s Un Globe Lace👍👍 liked it
                             ...                        
995    Clarks - Mens Un Larvik Top2 Boots, 10 UK, Gre...
996    Lethato Men’s Modern Classic Calfskin Leather ...
997    adidas NEMEZIZ 19.1 AG, Men's Soccer Shoe, Sig...
998    PUMA Women's Prowl Slip ON Walking Shoe, Black...
999    New Balance Girl's 515v1 Sneaker, Phantom/Neo ...
Length: 1000, dtype: object

In [ ]:
def preprocess(text):
  import re
  text=re.sub(r'[^a-zA-Z]',' ',text)
  text=re.sub(r'#\w+'," ",text)
  text=re.sub(r'@\w+'," ",text)
  return(text)

In [ ]:
preprocess("Lfin in3n5 56n6n6 -002 ")

'Lfin in n    n n       '

In [ ]:
df['text']=df['text'].apply(preprocess)

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
import sys

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df

,text,labels
0,Klasified Women s Transparent Clear Sneaker Sh...,5.0
1,Klasified Women s Transparent Clear Sneaker Sh...,5.0
2,Klasified Women s Transparent Clear Sneaker Sh...,5.0
3,GUESS Women s Bradly Gymnastics Shoe White ...,1.0
4,GUESS Women s Bradly Gymnastics Shoe White ...,5.0
...,...,...
3502,Kids Mesh Clogs Breathable Garden Shoes Toddle...,5.0
3503,Kids Mesh Clogs Breathable Garden Shoes Toddle...,5.0
3504,Kids Mesh Clogs Breathable Garden Shoes Toddle...,1.0
3505,Kids Mesh Clogs Breathable Garden Shoes Toddle...,4.0


In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

In [ ]:
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt

In [ ]:
model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 386, 120)          60000     
                                                                 
 spatial_dropout1d (SpatialD  (None, 386, 120)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 176)               209088    
                                                                 
 dense (Dense)               (None, 5)                 885       
                                                                 
Total params: 269,973
Trainable params: 269,973
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
y=pd.get_dummies(df['labels'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

In [ ]:
batch_size=32
history=model.fit(X_train, y_train, epochs = 3, batch_size=batch_size, verbose = 'auto')

Epoch 1/3
 2/77 [..............................] - ETA: 4:40 - loss: 1.6026 - accuracy: 0.4531

In [ ]:
test=pd.DataFrame(test_data,columns=['text'])

In [ ]:
df.shape

In [ ]:
df['text']=df['text'].apply(preprocess).apply(preprocess)

In [ ]:
test_data=tokenizer.texts_to_sequences(df['text'])
test_data_padded=pad_sequences(test_data,padding='post',maxlen=386)

In [ ]:
test_data_padded

array([[ 13,   4,  23, ...,   0,   0,   0],
       [ 13,   4,  23, ...,   0,   0,   0],
       [ 13,   4,  23, ...,   0,   0,   0],
       ...,
       [107, 404, 334, ...,   0,   0,   0],
       [107, 404, 334, ...,   0,   0,   0],
       [107, 404, 334, ...,   0,   0,   0]], dtype=int32)

In [ ]:
y_pred=model.predict(test_data_padded)

In [ ]:
len(y_pred),len(test)

3500

In [ ]:
submissions['label'] = y_pred
submissions.to_csv('/content/submission.csv', index=False)

array([[0.01542152, 0.00869817, 0.01612915, 0.06618759, 0.8935636 ],
       [0.01542153, 0.00869817, 0.01612915, 0.06618759, 0.8935636 ],
       [0.01542152, 0.00869817, 0.01612916, 0.06618759, 0.8935636 ],
       ...,
       [0.01542152, 0.00869817, 0.01612915, 0.06618759, 0.8935636 ],
       [0.01542152, 0.00869817, 0.01612915, 0.06618759, 0.8935636 ],
       [0.01542152, 0.00869817, 0.01612915, 0.06618759, 0.8935636 ]],
      dtype=float32)